# RA challenge - training and prediction
This notebook was used to train all the models used in the RA challenge,
to test performance, and to generate the Docker directory used to build the submitted image.
This should be run in a Jupyter fastai environment - the versions used in the challenge
were fastai 1.0.55, torch 1.1.0, and torchvision 0.3.0.
Note models trained and exported with these versions may be incompatible with later versions.

### How to train the models and generate the Docker image starting from scratch:
There are three types of models:
- orientation (predict which of the 8 possible 90-degree orientations an x-ray image is in)
- joint location (prediction where the joints used in RA scoring are on an image)
- RA label prediction (predict joint narrowing and erosion scores)

Each model uses the predictions of the previous ones.
We start by predicting the orientation; this is then used to rotate/flip all images to the same
orientation before feeding into the next models.
This allows us to **merge left and right images** into the same models.
We then predict joint locations and cut out a sub-image of the predicted joints.
This is fed into the RA label prediction models.
This allows us to **merge different joints** into the same models -
for example, to train one model to predict a joint erosion score for any finger joint.

To train the models and generate the Docker image starting from scratch, do the following:
- Set imDirPath to the directory containing the training images and training.csv file (see code cell 3). Make sure there's an empty dock/ subdirectory under imDirPath for Docker related files.
- Run the notebook up to the cell containing mark1.
- Run the four sections of code between mark3 and mark4 - this will train and save/export the orientation and joint location models.
- Restart the notebook and run up to the cell containing mark2. Then run the next two code cells to train all the RA label prediction models with current hyperparameter settings and update the dock/ subdirectory.
- Use the Dockerfile and other files in the dock/ subdirectory to build the Docker image.

### How to test performance and try out different hyperparameter settings:
Do the first two steps above to set up the orientation and joint location models.
Then, restart and run the notebook up to mark2.
You can then add and run new cells to test cross-validation performance with different settings. For some examples, see the section **Examples of performance testing** at the end of this notebook

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Setting up paths and auto-generated scripts

In [ ]:
running_exported_scripts = set()
# Indicate we're running from this notebook, not from one of
# the Python scripts that can be auto-generated from it.

In [ ]:
# export for_prediction for_joints
# Includes this cell in the auto-generated Python scripts
# for prediction and the joint marking GUI.

# import needed libraries:
import collections
import cv2
import itertools
import random

from fastai import *
from fastai import basic_train
from fastai.vision import *
import fastprogress

import faiutils

# set paths up for the notebook environment:
imDirPath = Config().data_path()/'ratrain2'
trainedModelPath = outDirPath = imDirPath
mainCsvFName = 'training.csv'
if ('for_prediction' in running_exported_scripts
        and 'for_prediction_test' not in running_exported_scripts) :
    # set paths up for the submitted Docker image:
    imDirPath = Path('/test')
    trainedModelPath = Path('/')
    outDirPath = Path('/output')
    mainCsvFName = 'template.csv'

# auto-generate some scripts from this notebook:
if not running_exported_scripts :
    # standalone script to predict the test set, used for submit:
    faiutils.exportFromNotebook('ra.ipynb',
                    ['for_prediction','stepAllNew'], 'rapredAllNew.py')
     # standalone script to predict the training set, used for local test:
    faiutils.exportFromNotebook('ra.ipynb',
                    ['for_prediction','stepAllNew','for_prediction_test'], 'rapredtestAllNew.py')
     # script for import to the GUI used to mark joints:
    faiutils.exportFromNotebook('ra.ipynb',
                    ['for_joints'], 'rajoints.py')
    faiutils.modelDirPath = modelDirPath = imDirPath/'models'
else :
    # turn off progress bar when running from auto-generated scripts
    fastprogress.fastprogress.NO_BAR = True
#     (basic_train.master_bar, basic_train.progress_bar
#         ) = fastprogress.force_console_behavior()

In [ ]:
# the imports below aren't required for prediction:

import ipywidgets as widgets
# from IPython.core.debugger import set_trace
from IPython.display import display #, Javascript

In [ ]:
imDirPath

## Data checking and massage

In [ ]:
# export for_prediction for_joints

# Functions to create lists of all individual feature labels:

# Master lists of labels. XX is a placeholder that can be substituted with either J or E.
footLabels = [f'F_mtp_XX__{suff}' for suff in ['5','4','3','2','1','ip']]
handTopLabels = [f'H_pip_XX__{suff}' for suff in '5432']
handMidLabels = [f'H_mcp_XX__{suff}' for suff in '54321'] + ['H_mcp_E__ip']
wrist1_Labels = ['H_wrist_J__cmc5', 'H_wrist_J__cmc4', 'H_wrist_J__cmc3']
wrist2_Labels = ['H_wrist_J__mna', 'H_wrist_J__radcar', 'H_wrist_J__capnlun']
wrist3_Labels = ['H_wrist_E__nav', 'H_wrist_E__mul', 'H_wrist_E__mc1']
wrist4_Labels = ['H_wrist_E__ulna', 'H_wrist_E__lunate', 'H_wrist_E__radius']
handBotLabels = wrist1_Labels + wrist2_Labels + wrist3_Labels + wrist4_Labels

def allMarkedJointLabels(HOrF) :
    """
    List of all labels that have been marked on the XRays for hands, feet, or both.
    This is a combination of the master lists above, except that the wrist labels
    are consolidated into 4 points per wrist to save time on marking.
    """
    if HOrF == 'F' :
        return footLabels
    elif HOrF == 'H' :
        return handTopLabels + handMidLabels + [f'H_wrist{suff}' for suff in '1234']
    elif HOrF == 'B' :
        return allMarkedJointLabels('F')+allMarkedJointLabels('H')
def initJointDF(df,HOrF) :
    "Initialize a dataframe for marking joint points."
    df = df[['Patient_ID']].copy()
    for label in allMarkedJointLabels(HOrF) :
        df[label+'x'] = -1.0
        df[label+'y'] = -1.0
    return df

markedLabelFrac = {}
for label in footLabels :
    markedLabelFrac[label] = 0.125
markedLabelFrac['F_mtp_XX__1'] = markedLabelFrac['F_mtp_XX__ip'] = 0.15
for label in handTopLabels :
    markedLabelFrac[label] = 0.125
for label in handMidLabels :
    markedLabelFrac[label] = 0.125
markedLabelFrac['H_mcp_XX__1'] = markedLabelFrac['H_mcp_E__ip'] = 0.15
markedLabelFrac['H_wrist1'] = 0.25
markedLabelFrac['H_wrist2'] = 0.25
markedLabelFrac['H_wrist3'] = 0.25
markedLabelFrac['H_wrist4'] = 0.25
assert (set(markedLabelFrac.keys()) == set(allMarkedJointLabels('B'))
       ), "marked label fraction list mismatch"

# construct a map from all labels to the corresponding marked joint label
labelToMarkedLabel = {}
for labels,markedLabel in [(wrist1_Labels, 'H_wrist1'), (wrist2_Labels, 'H_wrist2'),
                           (wrist3_Labels, 'H_wrist3'), (wrist4_Labels, 'H_wrist4')] :
    for label in labels :
        labelToMarkedLabel[label] = markedLabel
for HOrF in 'HF' :
    for markedLabel in allMarkedJointLabels(HOrF) :
        if 'XX' in markedLabel :
            labelToMarkedLabel[markedLabel.replace('XX','J')] = markedLabel
            labelToMarkedLabel[markedLabel.replace('XX','E')] = markedLabel
        elif '_J_' in markedLabel or '_E_' in markedLabel :
            labelToMarkedLabel[markedLabel] = markedLabel

# Functions that add a L or R prefix to the labels above, and substitute XX with E or F
# to generate the full set of labels in the RA dataset.
def add_LR_EJ(LOrR,EOrJ,labelList=[]) :
    labelList = [label.replace('XX',EOrJ) for label in labelList]
    return [LOrR+label for label in labelList
            if '_'+EOrJ+'__' in label]
FootLabelList = partial(add_LR_EJ, labelList=footLabels)
HandTopList = partial(add_LR_EJ, labelList=handTopLabels)
HandMidList = partial(add_LR_EJ, labelList=handMidLabels)
HandBotList = partial(add_LR_EJ, labelList=handBotLabels)
def HandLabelList(LOrR,EOrJ) :
    return (HandTopList(LOrR,EOrJ) + HandMidList(LOrR,EOrJ) + HandBotList(LOrR,EOrJ))

assert set(FootLabelList('','J') + FootLabelList('','E')
           + HandLabelList('','J') + HandLabelList('','E')) == set(labelToMarkedLabel.keys()
        ), "label list mismatch"

def LabelList(LOrR,EOrJ,HOrF) :
    if HOrF=='H' :
        return HandLabelList(LOrR,EOrJ)
    elif HOrF=='I' :
        return HandTopList(LOrR,EOrJ)
    elif HOrF=='J' :
        return HandMidList(LOrR,EOrJ)
    elif HOrF=='K' :
        return HandBotList(LOrR,EOrJ)
    elif HOrF=='F' :
        return FootLabelList(LOrR,EOrJ)
    else :
        raise Exception('unknown label type '+str(HOrF))

# Do a couple of checks that we got all those names right:

print('checking total scores ... ',end='')
maxScoreForLabel = {}
for label in (FootLabelList('L','J')+FootLabelList('R','J')
              + HandLabelList('L','J')+HandLabelList('R','J')) :
    maxScoreForLabel[label] = 4
for label in FootLabelList('L','E')+FootLabelList('R','E') :
    maxScoreForLabel[label] = 10
for label in HandLabelList('L','E')+HandLabelList('R','E') :
    maxScoreForLabel[label] = 5
assert sum(maxScoreForLabel.values())==448, "total max score should be 448"

print('checking table columns ... ',end='')
mainDF = pd.read_csv(imDirPath/mainCsvFName)
mainColumns = set(mainDF.columns)
for label in maxScoreForLabel.keys() :
        assert label in mainColumns, "missing data column "+label
        if not running_exported_scripts :
            assert (mainDF[label].min()>=0
                    and mainDF[label].max()<=maxScoreForLabel[label]),(
                    "value out of range for "+label)
print('done')

def clipPredictions(df) :
    "Clip predicted label values in a dataframe to the valid range for each label."
    for col,maxScore in maxScoreForLabel.items() :
        df[col].clip(0,maxScore,inplace=True)

In [ ]:
# export for_prediction for_joints

def printCounts(label,df) :
    "Do a quick check of label distribution."
    labelsWithCounts = sorted(collections.Counter(df[label]).items())
    labelsWithCumCounts, cumCount = [],0
    for v,count in labelsWithCounts :
        cumCount += count
        labelsWithCumCounts.append((v,count,cumCount))
    print('counts for '+label+':',labelsWithCumCounts)
printCounts('LF_mtp_J__5',mainDF)

In [ ]:
def calcValidationSliceMap(idList, seed=3141) :
    "Split the training set into n possible validation sets."
    idList = list(idList)
    random.Random(seed).shuffle(idList)
    return dict((id,i) for i,id in enumerate(idList))

validationSliceMap = calcValidationSliceMap(mainDF['Patient_ID'])

In [ ]:
# validationSliceMap
# {'UAB024': 0,
#  'UAB278': 1,
#  'UAB229': 2,
#  'UAB405': 3,
#  'UAB277': 4,
#  'UAB451': 0,
# ...

In [ ]:
# export for_prediction for_joints

def dupLR(df) :
    "Duplicate the training dataframe, adding 'L' and 'R' to one of each duplicated patient ID."
    nPatients = len(df)
    print('duplicating',nPatients,'rows ... ',end='')
    dupDF = pd.concat([df,df]).reset_index(drop=True)
    dupDF['Patient_ID'] += (nPatients*['L']+nPatients*['R'])
    print('now',len(dupDF),'rows')
    return dupDF

def patientIDToIndex(df,patientID) :
    return df[df.Patient_ID==patientID].index[0]

In [ ]:
# export for_joints

dupDF = dupLR(mainDF)

pd.concat([dupDF[0:2],dupDF[len(mainDF):len(mainDF)+2]])

In [ ]:
# export for_prediction for_joints

def splitDupCol(dupDF,newCol,convertToFloat=False) :
    """
    Creates a new column with L/R removed and, for each duplicated patient ID with L or R added,
    fills the new columns with the values from the L or R column in that row.

    So, after calling dupLR and splitDupCol we go from:
      PatientID  Lcol Rcol
      Patient1    8    9
    in the original main dataframe to:
      PatientID  Lcol Rcol  col
      Patient1L   8    9     8
      Patient1R   8    9     9
    """
    dupDF[newCol] = dupDF.apply(
        lambda row : row[row['Patient_ID'][-1]+newCol],
        axis=1)
    if convertToFloat :
        dupDF[newCol] = dupDF[newCol].astype(float)
    printCounts(newCol,dupDF)
def splitAllDupCols(dupDF) :
    "Call splitDupCol for all single joint feature columns."
    for EOrJ,HOrF in itertools.product("EJ","HF") :
        for newCol in LabelList('',EOrJ,HOrF) :
            splitDupCol(dupDF,newCol)

In [ ]:
# export for_joints

splitAllDupCols(dupDF)

In [ ]:
dupDF.loc[[0,1,len(mainDF),len(mainDF)+1]][
    ['Patient_ID','LF_mtp_J__4','RF_mtp_J__4','F_mtp_J__4',
     'LF_mtp_J__5','RF_mtp_J__5','F_mtp_J__5']]

## Training

In [ ]:
# export for_prediction for_joints

# Hyperparameters that affect training.
# These are passed to the training functions (setupTraining, etc) using **
defTrainingHPars = dict(
    bs = 32,
    imSize = 256,
    trSeed=3141,
    # trLabel = 'H_pip_E__5', # label or list of labels to train
    # trHOrF = 'H',      # hand or foot
    trJoints = None,
    applyClahe = False,
    myRemap=False, meanChange=0.05, stdDevChange=0.025,
    vertStart=0.0, vertFrac=1.0, horizStart=0.0, horizFrac=1.0,
    doOtsu=False,
    gBlur=0, cv2Interp=False,
    resizeToSize=None, returnNumpy=False,
    topCropThresh=None,
    padFrac=0.05,
    arch=models.resnet34,
    wd=0.01,
    max_rotate=10.0,
    max_lighting=0.2,
    max_warp=0.2,
    max_zoom=1.1,
    do_flip=False, flip_vert=False,
    borderType=cv2.BORDER_CONSTANT,
    lin_ftrs=None,
    ps=0.5,
    nExtraCopiesFunc=None, # Function on row that returns number of extra copies
                           # to make of that row for training.
                           # ex: lambda row : 3 if row['Overall_Tol']>10 else 0
    customHead=None, # if not None, should be a function returning
                     # a custom head to supply to cnn_learner
    addLayer=None, # if not None, should be a function returning
                   # a layer to add to the NN, ex. for clipping output
    cutLabel=None,
)
trainingHPars = dict(defTrainingHPars)

## Special-purpose ImageList for the RA XRay images
Flips the right-side images horizontally,
and applies other transforms and modifications as specified by trainingHPars.

In [ ]:
# export for_prediction for_joints

raGuiDispImSize=512

def cv2fai(im) :
    #im = 255-im
    if len(im.shape) == 2 :
        im = np.repeat(im[:,:,np.newaxis],3,axis=2)
    #print(im.shape)
    #im[:,:,0] = np.linspace(50,200,im.shape[1])
    return Image(pil2tensor(im,dtype=np.float32).div_(255))
def padIm(im,frac=0.1) :
    height,width = im.shape
    maxDim = max(height,width)
    pad = int(maxDim*frac)
    xPos = pad + int((maxDim-width)/2)
    yPos = pad + int((maxDim-height)/2)
    res = cv2.copyMakeBorder(im,yPos,maxDim+2*pad-(yPos+height),
                             xPos,maxDim+2*pad-(xPos+width),
                             trainingHPars['borderType'])
    return res
def findHistPosAbove(hist,thresh,pos,inc) :
    while hist[pos] < thresh :
        pos += inc
    return pos
def remapChan(im,outIm,low,high,toLow,toHigh) :
    outIm[:,:] = im
    outIm /= 255.0
    outIm.clip(low,high,outIm)
    outIm -= low
    outIm *= ((toHigh-toLow)/(high-low))
    outIm += toLow
def myRemap(im, imStats=imagenet_stats, stdDevMult=1.5) :
    hist = [int(h[0]) for h in cv2.calcHist([im],[0],None,[256],[0,256])]
    low = findHistPosAbove(hist,20,0,1)/255.0
    high = findHistPosAbove(hist,20,255,-1)/255.0
    outIm = np.zeros(im.shape,np.float32)
    means,stdDevs = imStats
    chMean,chStdDev = means[1],stdDevs[1]
    chMean += random.uniform(-trainingHPars['meanChange'],trainingHPars['meanChange'])
    chStdDev += random.uniform(-trainingHPars['stdDevChange'],trainingHPars['stdDevChange'])
    remapChan(im,outIm, low, high,
                  max(0.0,chMean-stdDevMult*chStdDev),
                  min(1.0,chMean+stdDevMult*chStdDev))
    outIm *= 255.0
    im[:,:] = outIm
def cropIm(im,vertStart,vertFrac,horizStart,horizFrac) :
    height,width = im.shape
    return im[round(height*vertStart):round(height*(vertStart+vertFrac)),
              round(width*horizStart):round(width*(horizStart+horizFrac))]
def isBorder(diffs,thresh) :
    return len(diffs)>10 and sorted(diffs)[-10]>=thresh
def findTopBorder(im,thresh) :
    height,width = im.shape
    iStart = int(height*0.02)
    for i in range(iStart,int(height*0.4)) :
        diffs = np.abs((im[iStart]-im[i]).astype(np.int8))
        if isBorder(diffs,thresh) :
            return i
    return int(height*0.05)
def cropBorders(im) :
    if trainingHPars['topCropThresh'] is not None :
        im = im[findTopBorder(im,trainingHPars['topCropThresh']):]
#     left = findLeftBorder(im,thresh)
#     im = im[:,left:]
#     right = findRightBorder(im,thresh)
    return im

def getRowJointPoints(jdf,ind) :
    jpRow = jdf.loc[ind]
    jpRow = [p*raGuiDispImSize for p in jpRow[1:]]  # rescale [0.0..1.0] to [0..raGuiDispImSize]
    return tensor(jpRow).reshape((len(jpRow)//2,2))  # split into y,x coordinate pairs
def getPatientJointPoints(jdf,patientID) :
    return getRowJointPoints(jdf,patientIDToIndex(jdf,patientID))
def getJointPoints(il,fPath) :
    return getPatientJointPoints(il.jdf,os.path.basename(fPath))

# show marked image points for debugging:
def showPoints(il,patientID,scaledPts=None,figsize=(9,9)) :
    if isinstance(patientID,int) :
        ind = patientID
    else :
        ind = patientIDToIndex(il.jdf,patientID)
    img = il[ind]
    if scaledPts is not None :
        if len(scaledPts.shape) == 1 :
            scaledPts = scaledPts.reshape((scaledPts.shape[0]//2,2))
        jpRow = ((scaledPts+1.0)/2.0)*raGuiDispImSize
    else :
        jpRow = getRowJointPoints(il.jdf,ind)
    jpRow = jpRow.float()
    #print(jpRow)
    #print(FlowField(torch.Size([raGuiDispImSize,raGuiDispImSize]),jpRow))
    points = ImagePoints(FlowField(torch.Size([raGuiDispImSize,raGuiDispImSize]),jpRow))
    #print('image size:',img.size)
    img.show(y=points,figsize=figsize)

def getScaledChunkBounds(imSize,coord,frac=0.1,scaling='none') :
    if scaling=='0..1' or scaling=='-1..1' :
        if scaling == '-1..1' :
            coord = (coord+1.0)/2.0
        coord = int(coord*imSize)
    chunkSize = int(imSize*frac/2.0)
    coordStart = max(0,coord-chunkSize)
    coordEnd = coordStart+2*chunkSize
    if coordEnd>imSize :
        coordEnd = imSize
        coordStart = coordEnd-2*chunkSize
    return coordStart,coordEnd
def getImChunk(im,x,y,frac=0.1,scaling='none') :
    imSize = im.shape[0]
    xStart,xEnd = getScaledChunkBounds(imSize,x,frac,scaling)
    yStart,yEnd = getScaledChunkBounds(imSize,y,frac,scaling)
    return im[yStart:yEnd,xStart:xEnd]

# draw box around image points for debugging:
def markImChunk(im,x,y,frac=0.1,scaling='none') :
    imSize = im.shape[0]
    markSize = 3
    xStart,xEnd = getScaledChunkBounds(imSize,x,frac,scaling)
    yStart,yEnd = getScaledChunkBounds(imSize,y,frac,scaling)
    im[yStart:yEnd,xStart:xStart+markSize] = 255
    im[yStart:yEnd,xEnd-markSize:xEnd] = 255
    im[yStart:yStart+markSize,xStart:xEnd] = 255
    im[yEnd-markSize:yEnd,xStart:xEnd] = 255
def markImChunks(im,pts,frac=0.1,scaling='-1..1') :
    if pts is not None :
        if len(pts.shape) == 1 :
            pts = pts.reshape((pts.shape[0]//2,2))
        for y,x in pts :
            markImChunk(im,x.item(),y.item(),frac,scaling)

def addYX(labels) :
    return list(itertools.chain.from_iterable([label+'y',label+'x'] for label in labels))

def doFlipMod(im,imageFlipMod) :
    doFlip,nRot = imageFlipMod[0],imageFlipMod[1]
    if nRot == '1' :
        im = cv2.rotate(im,cv2.ROTATE_90_CLOCKWISE)
    elif nRot == '2' :
        im = cv2.rotate(im,cv2.ROTATE_180)
    elif nRot == '3' :
        im = cv2.rotate(im,cv2.ROTATE_90_COUNTERCLOCKWISE)
    if doFlip == 'F' :
        im = im[::-1,:]
    return im
def undoFlipMod(im,imageFlipMod) :
    doFlip,nRot = imageFlipMod[0],imageFlipMod[1]
    if doFlip == 'F' :
        im = im[::-1,:]
    if nRot == '1' :
        im = cv2.rotate(im,cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif nRot == '2' :
        im = cv2.rotate(im,cv2.ROTATE_180)
    elif nRot == '3' :
        im = cv2.rotate(im,cv2.ROTATE_90_CLOCKWISE)
    return im
possibleFlipMods = ['F0','F1','F2','F3','N0','N1','N2','N3']
def makeFlipModDF(df,nRepeats=1) :
    imMods = nRepeats*list(itertools.chain.from_iterable(
                        len(df)*[imMod] for imMod in possibleFlipMods))
    dfPatients = df[['Patient_ID']]
    dfPatients = (nRepeats*8)*[dfPatients]
    df = pd.concat(dfPatients).reset_index(drop=True)
    df['Patient_ID'] += ['$$' + imMod for imMod in imMods]
    df['imMod'] = imMods
    return df


class RAImageList(ImageList) :
    def open(self, fn) :
        # print(fn)
        forceHOrF = None
        resMarkedLabel = trainingHPars['cutLabel']
        if '*' in fn :
            fn,resMarkedLabel = fn.split('*')
            if 'XX' not in resMarkedLabel :
                resMarkedLabel = labelToMarkedLabel[resMarkedLabel]
            forceHOrF = resMarkedLabel[0]
        if resMarkedLabel is not None :
            resFrac = markedLabelFrac[resMarkedLabel]
            predJDF = trainingHPars['predictedJointsDF']
            ind = patientIDToIndex(predJDF,os.path.basename(fn))
            resPredictedX = predJDF.at[ind,resMarkedLabel+'x']
            resPredictedY = predJDF.at[ind,resMarkedLabel+'y']
        imageFlipMod = None
        if '$$' in fn :
            fn,imageFlipMod = fn.split('$$')
            flipModFunc = doFlipMod
        else :
            predODF = trainingHPars.get('predictedOrientDF')
            if predODF is not None :
                ind = patientIDToIndex(predODF,os.path.basename(fn))
                imageFlipMod = predODF.at[ind,trainingHPars['trHOrF']+'Orient']
                flipModFunc = undoFlipMod
        LOrR = fn[-1]
        fn = fn[:-1]
        if fn[-1]=='-' :
            fn = fn[:-1]
        fn += ('-'+LOrR+trainingHPars['trHOrF']+'.jpg')
        if os.path.isabs(fn) :
            imPath = fn
        else :
            imPath = str(imDirPath/fn)
        #print('fp',imPath)
        if forceHOrF :
            im = cv2.imread(re.sub('..jpg',forceHOrF+'.jpg',imPath),0)
        else :
            im = cv2.imread(imPath,0)
        if LOrR=='R' :
            im = im[:,::-1]
        if trainingHPars['applyClahe'] :
            if not hasattr(self,'clahe') :
                self.clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16,16))
        if trainingHPars['trJoints'] is not None :
            if not hasattr(self,'jdf') :
                self.jdf = pd.read_csv(trainingHPars['trHOrF']+'joints.csv')
                markedLabelsYX = addYX(trainingHPars['trJoints'])
                self.jdf = self.jdf[['Patient_ID']+markedLabelsYX]
        if trainingHPars['doOtsu'] :
            blur = cv2.GaussianBlur(im, (5,5), 0)
            _,im = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        else :
            im = cropBorders(im)
            im = cropIm(im,trainingHPars['vertStart'],trainingHPars['vertFrac'],
                        trainingHPars['horizStart'],trainingHPars['horizFrac'])
            im = padIm(im,trainingHPars['padFrac'])
            if imageFlipMod is not None :
                im = flipModFunc(im,imageFlipMod)
            if resMarkedLabel is not None :
                im = getImChunk(im, resPredictedX, resPredictedY, frac=resFrac, scaling='-1..1')
            if trainingHPars['myRemap'] :
                myRemap(im)
            if trainingHPars['applyClahe'] :
                im = self.clahe.apply(im)
            markImChunks(im,trainingHPars.get('debugMarkPts'),0.1)
        gBlurSize = trainingHPars['gBlur']
        if gBlurSize and im.shape[0]>trainingHPars['imSize']:
            im = cv2.GaussianBlur(im,(gBlurSize,gBlurSize),0)
        resizeToSize = trainingHPars['resizeToSize']
        if resizeToSize is not None :
            if isinstance(resizeToSize,int) :
                resizeToSize = (resizeToSize,resizeToSize)
            im = cv2.resize(im,resizeToSize)
        elif trainingHPars['cv2Interp'] :
            fromSize = im.shape[0]
            toSize = trainingHPars['imSize']
            interpType = cv2.INTER_AREA if fromSize>toSize else cv2.INTER_CUBIC
            im = cv2.resize(im,(toSize,toSize),interpolation=interpType)
        if trainingHPars['returnNumpy'] :
            return im
        return cv2fai(im)
def RAImShow(fn,withPars=None) :
    global trainingHPars
    if withPars is not None :
        trainingHPars = withPars
    return RAImageList.from_folder(imDirPath).open(fn)
def RAProcAll(toDir,withPars=None) :
    global trainingHPars
    if withPars is not None :
        trainingHPars = withPars
    destDirPath = Config().data_path()/toDir
    if destDirPath.exists() :
        print(destDirPath,'already exists!')
        return
    imNames = [imName.name for imName in imDirPath.ls()]
    imNames = [imName[:-5].replace('-','')
               for imName in imNames
               if imName.endswith(trainingHPars['trHOrF']+'.jpg')]
    il = RAImageList.from_df(pd.DataFrame(imNames),imDirPath)
    destDirPath.mkdir()
    for fp,im in zip(il.items,il) :
        imFName = os.path.basename(fp[:-1]+'-'+fp[-1]+trainingHPars['trHOrF']+'.jpg')
        print(fp,'->',destDirPath/imFName,im)
        im.save(destDirPath/imFName)

## Various experiments with customizing the architecture:

In [ ]:
# export for_prediction

# The default cnn_learner head:
# BatchNorm1d          [1024]               2,048      True      
# ______________________________________________________________________
# Dropout              [1024]               0          False     
# ______________________________________________________________________
# Linear               [512]                524,800    True      
# ______________________________________________________________________
# ReLU                 [512]                0          False     
# ______________________________________________________________________
# BatchNorm1d          [512]                1,024      True      
# ______________________________________________________________________
# Dropout              [512]                0          False     
# ______________________________________________________________________
# Linear  

def customHead1(endDepth,endSize,hiddenSize=256) :
    return nn.Sequential(
        nn.Conv2d(endDepth,hiddenSize,1),
        nn.ReLU(),
        nn.BatchNorm2d(hiddenSize),
        nn.Conv2d(hiddenSize,2,1),
        Flatten(),
        nn.Linear(endSize*endSize*2,256),
        nn.ReLU(),
        nn.BatchNorm1d(256),
        nn.Linear(256,2)
    )
class scaledSigma(nn.Module) :
    def __init__(self, lower, upper, slop=None) :
        """scale output to lower-slop .. upper+slop using sigmoid"""
        super(scaledSigma,self).__init__()
        if slop is None :
            slop = 0.1*(upper-lower)
        self.yLow = lower-slop
        self.yFactor = 2*slop + (upper-lower)
        self.lower, self.upper, self.slop = lower,upper,slop
    def forward(self, input) :
        return self.yLow + self.yFactor*torch.sigmoid(input)
class clipOutput(nn.Module) :
    def __init__(self, lower, upper) :
        """clip output to lower .. upper"""
        super(clipOutput,self).__init__()
        self.lower, self.upper = lower, upper
    def forward(self, input) :
        return torch.clamp(input,self.lower,self.upper)

## Low-level functions for training and prediction:

In [ ]:
# export for_prediction

def clearCurGlobals() :
    global il,data,learn
    il = data = learn = None
def setupTraining(valSliceNo=0, nExtraCopiesFunc=None,
                  label_cls=FloatList, metrics=None, df=None, nCrossValSlices=5, **kwargs) :
    global trainingHPars,il,data,learn
    if df is None :
        df = dupDF
    print(f'CV slice {valSliceNo}/{nCrossValSlices}')
    trainingHPars = dict(defTrainingHPars,**kwargs)
    clearCurGlobals()
    if valSliceNo is not None :
        df = df.copy()
        df['isValid'] = df['Patient_ID'].apply(lambda x :
                (validationSliceMap[x.split('*')[0].split('$$')[0][:-1]] % nCrossValSlices) == valSliceNo)
        notInValidSet = ~(df['isValid'])
    if nExtraCopiesFunc is not None :
        nExtraCopies = df.apply(nExtraCopiesFunc, axis=1)
        i = nExtraRows = 0
        extraRowsList = []
        while True :
            if valSliceNo is not None :
                extraRows = df[(nExtraCopies>i) & notInValidSet]
            else :
                extraRows = df[(nExtraCopies>i)]
            if len(extraRows) == 0 :
                break
            extraRowsList.append(extraRows)
            nExtraRows += len(extraRows)
            i += 1
        print('adding',nExtraRows,'extra rows ... ',end='')
        df = pd.concat([df]+extraRowsList).reset_index(drop=True)
        print(len(df),'total rows')
    il = RAImageList.from_df(df,imDirPath)
    img = il[0]  # will initialize il.clahe and il.jdf if needed
    if valSliceNo is None :
        ilsp = il.split_none()
    else :
        ilsp = il.split_from_df('isValid')
    doingJointTraining = (trainingHPars['trJoints'] is not None)
    if doingJointTraining :
        ilspLab = ilsp.label_from_func(partial(getJointPoints,il),label_cls=PointsLabelList)
    else :
        ilspLab = ilsp.label_from_df(trainingHPars['trLabel'],label_cls=label_cls)
    tfms = get_transforms(do_flip=trainingHPars['do_flip'],flip_vert=trainingHPars['flip_vert'],
                         max_rotate=trainingHPars['max_rotate'],
                         max_lighting=trainingHPars['max_lighting'],
                         max_warp=trainingHPars['max_warp'],
                         max_zoom=trainingHPars['max_zoom'],
                         )
    data = ilspLab.transform((tfms[0],[]) if doingJointTraining else tfms,
                             tfm_y=doingJointTraining,
                             size=trainingHPars['imSize']
                            ).databunch(bs=trainingHPars['bs'],
                                        num_workers=0,
                            ).normalize(imagenet_stats)
    faiutils.randomSeedForTraining(trainingHPars['trSeed'])
    if trainingHPars['customHead'] is not None :
        custom_head = trainingHPars['customHead']()
    else :
        custom_head = None
    learn = cnn_learner(data,trainingHPars['arch'],
                        wd=trainingHPars['wd'],
                        lin_ftrs=trainingHPars['lin_ftrs'],
                        ps=trainingHPars['ps'],
                        custom_head=custom_head,
                        metrics=metrics,
                       )
    if trainingHPars['addLayer'] is not None :
        learn.model[1].add_module('addLayer',trainingHPars['addLayer']())
    print('loss',learn.loss_func)
    return learn,data
def tryTraining(nEpochs=20, lr=5e-3, **kwargs) :
    setupTraining(**kwargs)
    learn.fit_one_cycle(nEpochs,lr)
    val_losses = learn.recorder.val_losses
    # print(val_losses)
    if val_losses and val_losses[0] is not None :
        return {'final':val_losses[-1], 'min':min(val_losses)}
    return {}
def tryCrossVal(nEpochs=20, lr=5e-3, nCrossValSlices=5, **kwargs) :
    res = {'final':[], 'min':[]}
    for i in range(nCrossValSlices) :
        resI = tryTraining(nEpochs, lr, valSliceNo=i, nCrossValSlices=nCrossValSlices, **kwargs)
        print(resI)
        for k in res.keys() :
            res[k].append(resI[k])
    for k in res.keys() :
        res[k] = sum(res[k])/len(res[k])
    return res
def doFullTraining(exportFName=None, nEpochs=20, lr=5e-3, **kwargs) :
    print(exportFName,nEpochs,lr,kwargs)
    tryTraining(nEpochs, lr, valSliceNo=None, **kwargs)
    learn.export(exportFName,destroy=True)
    clearCurGlobals()
def predictUsingExportedModel(exportFDir, exportFName, df, testImDirPath, labelList, hPars) :
    global trainingHPars
    trainingHPars = dict(defTrainingHPars,**hPars)
    learn = load_learner(path=exportFDir, file=exportFName,
                         num_workers=0)
    learn.data.add_test(RAImageList.from_df(df, testImDirPath),
                        tfms=None, tfm_y=False)
    print('loaded batch size',learn.data.batch_size)
    #learn.data.batch_size = 4
    preds,_ = learn.get_preds(ds_type=DatasetType.Test)
    print('output shape',preds.shape)
    learn.destroy()
    del learn
    gc.collect()
    if labelList is not None :
        df[labelList] = preds
    return preds
# def predictUsingExportedModel2(exportFDir, exportFName, df, testImDirPath, labelList, hPars) :
#     global trainingHPars
#     trainingHPars = dict(defTrainingHPars,**hPars)
#     learn = load_learner(path=exportFDir, file=exportFName,
#                          test=RAImageList.from_df(df, testImDirPath),
#                          num_workers=0)
#     print('loaded batch size',learn.data.batch_size)
#     #learn.data.batch_size = 4
#     preds,_ = learn.get_preds(ds_type=DatasetType.Test)
#     print('output shape',preds.shape)
#     learn.destroy()
#     del learn
#     gc.collect()
#     if df is not None :
#         df[labelList] = preds
#     return preds

# Joint prediction

In [ ]:
# export for_prediction

# current hyperparameter settings for training joint location models:
JLocTrainPars = dict(defTrainingHPars,
                     resizeToSize=raGuiDispImSize,
                     max_rotate=5.0,max_zoom=1.05,max_warp=0.1,
                     wd=0.1,
                    )
FJLocTrainPars = dict(JLocTrainPars, trHOrF='F', trJoints=allMarkedJointLabels('F'))
HJLocTrainPars = dict(JLocTrainPars, trHOrF='H', trJoints=allMarkedJointLabels('H'))

JPModelLists = {
    'F': [('FJoint_40epoch_0_04_wd_0_1', allMarkedJointLabels('F'), FJLocTrainPars)],
    'H': [('HJoint_40epoch_0_04_wd_0_1', allMarkedJointLabels('H'), HJLocTrainPars)],
}
JPModelLists['B'] = JPModelLists['F']+JPModelLists['H']

def predictJoints(df,HOrF,predictedOrientDF=None) :
    predictedJointsDF = initJointDF(df,HOrF)
    for modelName,markedLabelList,hPars in JPModelLists[HOrF] :
        predictUsingExportedModel(trainedModelPath, modelName+'.pkl', predictedJointsDF,
                                  imDirPath, addYX(markedLabelList),
                                  dict(hPars, trJoints=None, predictedOrientDF=predictedOrientDF))
    return predictedJointsDF

# Orientation prediction

In [ ]:
# export for_prediction

# current hyperparameter settings for training orientation models:
OrientTrainPars = dict(defTrainingHPars,
                       label_cls=None, metrics=accuracy, trLabel='imMod', imSize=224,
                       wd=0.1,
                    )
FOrientTrainPars = dict(OrientTrainPars, trHOrF='F')
HOrientTrainPars = dict(OrientTrainPars, trHOrF='H')

OrientModelLists = {
    'F': [('FOrient_3epoch_0_01_wd_0_1', 'FOrient', FOrientTrainPars)],
    'H': [('HOrient_3epoch_0_01_wd_0_1', 'HOrient', HOrientTrainPars)],
}
OrientModelLists['B'] = OrientModelLists['F']+OrientModelLists['H']

def predictOrient(df,HOrF) :
    predictedOrientDF = df[['Patient_ID']].copy()
    for modelName,resLabel,hPars in OrientModelLists[HOrF] :
        preds = predictUsingExportedModel(trainedModelPath, modelName+'.pkl', predictedOrientDF,
                                          imDirPath, None, hPars)
        preds = [possibleFlipMods[pred.item()] for pred in preds.argmax(dim=1)]
        predictedOrientDF[resLabel] = preds
        print('orientations',resLabel,set(preds))
    return predictedOrientDF

In [ ]:
# orpredDF = predictOrient(dupDF,'B')
# set(orpredDF[:367]['HOrient']),set(orpredDF[:367]['FOrient']),set(orpredDF[367:]['HOrient']),set(orpredDF[367:]['FOrient'])
# orpredDF.at[366,'FOrient'],orpredDF.at[367,'HOrient']

# Using predicted joints for training and label prediction

In [ ]:
# export for_prediction

def combDFToPointDF(combDF,labels) :
    pointDFList = []
    if isinstance(labels,str) : labels = [labels]
    for label in labels :
        pointDFList.append(combDF[['Patient_ID']].copy())
        pointDFList[-1]['Patient_ID'] += ('*'+label)
        if 'XX' in label :
            pointDFList[-1]['resultJ'] = combDF[label.replace('XX','J')]
            pointDFList[-1]['resultE'] = combDF[label.replace('XX','E')]
        else :
            pointDFList[-1]['result'] = combDF[label]
    return pd.concat(pointDFList).reset_index(drop=True)
def insertPointDF(pointDF,combDF) :
    n = len(combDF)
    for pos in range(0,len(pointDF),n) :
        labelDF = pointDF[pos:pos+n].copy().reset_index(drop=True)
        curLabel = labelDF['Patient_ID'][0].split('*')[1]
        if 'XX' in curLabel :
            combDF[curLabel.replace('XX','J')] = labelDF['resultJ']
            combDF[curLabel.replace('XX','E')] = labelDF['resultE']
        else :
            combDF[curLabel] = labelDF['result']

def trainUsingPredictedJoints(trainingFunc, trLabel=None, **kwargs) :
    pointDF = combDFToPointDF(dupDF,trLabel)
    return trainingFunc(trLabel='result', df=pointDF,
                        predictedJointsDF=predictedDupDFJoints, **kwargs)
def trainUsingPredictedJoints2(trainingFunc, trLabel=None, **kwargs) :
    pointDF = combDFToPointDF(dupDF,trLabel)
    return trainingFunc(trLabel=['resultJ','resultE'], df=pointDF,
                        predictedJointsDF=predictedDupDFJoints, **kwargs)

def trainUsingCutJoint(trainingFunc, **kwargs) :
    return trainingFunc(predictedJointsDF=predictedDupDFJoints, **kwargs)


In [ ]:
# export for_prediction

def pearsonr(x,y) :
    # print(x.shape,y.shape)
    x,y = x.flatten(),y.flatten()
    meanx = torch.mean(x)
    meany = torch.mean(y)
    xm = x.sub(meanx)
    ym = y.sub(meany)
    rnum = xm.dot(ym)
    rden = torch.norm(xm,2) * torch.norm(ym,2)
    return rnum/rden
class PearsonR(Callback) :
    def on_epoch_begin(self,**kwargs) :
        self.preds = []
        self.target = []
    def on_batch_end(self, last_output, last_target, **kwargs) :
        self.preds.append(last_output.detach().clone())
        self.target.append(last_target.detach().clone())
    def on_epoch_end(self, last_metrics, **kwargs) :
        self.preds = torch.cat(self.preds)
        self.target = torch.cat(self.target)
        return add_metrics(last_metrics, pearsonr(self.preds, self.target).item())
    def __repr__(self) :
        return 'PearsonR()'

In [ ]:
# export for_prediction

# current hyperparameter settings for training RA label prediction models:
newHPars = dict(arch=models.densenet201,
                addLayer=partial(scaledSigma,0.0,4.0,0.4),
                imSize=224,
                wd=0.075,
                bs=24,
                nExtraCopiesFunc = lambda row : 0 if row[1]==0.0 else 1,
                nEpochs=30,
                lr=0.01,
                label_cls=FloatList,
                metrics=[PearsonR()],
                #myRemap=True,
                #applyClahe=True,
)

pointHPars = {
    'footJ' : dict(newHPars, trHOrF='F',
            trLabel=['F_mtp_J__5','F_mtp_J__4','F_mtp_J__3','F_mtp_J__2','F_mtp_J__1',
                     'F_mtp_J__ip'],
    ),
    'footE' : dict(newHPars, trHOrF='F',
            trLabel=['F_mtp_E__5','F_mtp_E__4','F_mtp_E__3','F_mtp_E__2','F_mtp_E__1',
                     'F_mtp_E__ip'],
            addLayer=partial(scaledSigma,0.0,10.0,0.5),
            nEpochs=50, lr=0.005, gBlur=3,
    ),
    'handTopJ' : dict(newHPars, trHOrF='H',
            trLabel=['H_pip_J__5','H_pip_J__4','H_pip_J__3','H_pip_J__2'],
    ),
    'handMidJ' : dict(newHPars, trHOrF='H',
            trLabel=['H_mcp_J__5','H_mcp_J__4','H_mcp_J__3','H_mcp_J__2','H_mcp_J__1'],
    ),
    'handTopAndMidE' : dict(newHPars, trHOrF='H',
            trLabel=['H_pip_E__5','H_pip_E__4','H_pip_E__3','H_pip_E__2',
                     'H_mcp_E__5','H_mcp_E__4','H_mcp_E__3','H_mcp_E__2','H_mcp_E__1',
                     'H_mcp_E__ip'],
            addLayer=partial(scaledSigma,0.0,5.0,0.5),
            nEpochs=50, lr=0.005, gBlur=3,
    ),
}

cutHPars = {
    'wrist1' : dict(newHPars, trHOrF='H', trLabel=wrist1_Labels, cutLabel='H_wrist1',
            nExtraCopiesFunc = lambda row : 1 if any(row[label]>0.0 for label in wrist1_Labels) else 0,
    ),
    'wrist2' : dict(newHPars, trHOrF='H', trLabel=wrist2_Labels, cutLabel='H_wrist2',
            nExtraCopiesFunc = lambda row : 1 if any(row[label]>0.0 for label in wrist2_Labels) else 0,
    ),
    'wrist3' : dict(newHPars, trHOrF='H', trLabel=wrist3_Labels, cutLabel='H_wrist3',
            nExtraCopiesFunc = lambda row : 1 if any(row[label]>0.0 for label in wrist3_Labels) else 0,
            addLayer=partial(scaledSigma,0.0,5.0,0.5),
    ),
    'wrist4' : dict(newHPars, trHOrF='H', trLabel=wrist4_Labels, cutLabel='H_wrist4',
            nExtraCopiesFunc = lambda row : 1 if any(row[label]>0.0 for label in wrist4_Labels) else 0,
            addLayer=partial(scaledSigma,0.0,5.0,0.5),
    ),
}

def fullTrainAllNew() :
    # train and export all models
    for name,hPars in pointHPars.items() :
        trainUsingPredictedJoints(doFullTraining,**dict(hPars, exportFName=name+'_model'+'.pkl'))
    for name,hPars in cutHPars.items() :
        trainUsingCutJoint(doFullTraining,**dict(hPars, exportFName=name+'_model'+'.pkl'))

## Updating the Docker directory

In [ ]:
def copyFToDocker(fromFName, dockerFileStrs, dockerDir, fromDir='.', toFName=None) :
    if toFName is None :
        toFName = fromFName
    shutil.copyfile(os.path.join(fromDir,fromFName),os.path.join(dockerDir,toFName))
    if dockerFileStrs is not None :
        dockerFileStrs.append(f'COPY {toFName} /{toFName}')

def updateDockerDir(dockerDir) :
    dockerFileStrs = [
"""FROM paperspace/fastai@sha256:81e470fb2e55509487541a4303f3f95a7d5e3c428d6ef925c9e50c039751e6ed

ENV PATH=$PATH:/opt/conda/bin/
ENV USER fastai
WORKDIR /
RUN conda uninstall -n fastai -y fastai pytorch torchvision
RUN conda install -n fastai -c pytorch -c fastai -y fastai=1.0.55 pytorch=1.1.0 torchvision=0.3.0
RUN conda install -n fastai -y opencv

# Required: Create /train /test and /output directories 
RUN mkdir /train
RUN mkdir /test
RUN mkdir /output
"""]
    for name,_,_ in JPModelLists['B'] :
        copyFToDocker(name+'.pkl',dockerFileStrs,dockerDir,str(trainedModelPath))
    for name,_,_ in OrientModelLists['B'] :
        copyFToDocker(name+'.pkl',dockerFileStrs,dockerDir,str(trainedModelPath))
    for name,_ in pointHPars.items() :
        copyFToDocker(name+'_model'+'.pkl',dockerFileStrs,dockerDir,str(trainedModelPath))
    for name,_ in cutHPars.items() :
        copyFToDocker(name+'_model'+'.pkl',dockerFileStrs,dockerDir,str(trainedModelPath))
    copyFToDocker('rapredAllNew.py',dockerFileStrs,dockerDir)
    copyFToDocker('faiutils.py',dockerFileStrs,dockerDir)
    copyFToDocker('runallnew.sh',dockerFileStrs,dockerDir,toFName='run.sh')
    copyFToDocker('build.sh',None,dockerDir)
    copyFToDocker('eval.sh',None,dockerDir)
    copyFToDocker('term.sh',None,dockerDir)
    dockerFileStrs.append("""
# Make model and runfiles executable 
RUN chmod 775 /run.sh

# This is for the virtualenv defined above, if not using a virtualenv, this is not necessary
RUN chmod 755 /root #to make virtualenv accessible to singularity user

# Required: define an entrypoint. run.sh will run the model for us, but in a different configuration
# you could simply call the model file directly as an entrypoint 
ENTRYPOINT ["/bin/bash", "/run.sh"]
""")
    with open(os.path.join(dockerDir,'Dockerfile'),'w') as f :
        f.write('\n'.join(dockerFileStrs))
    print('updated',dockerDir)

## Test set prediction

In [ ]:
# export for_prediction

def initPredSplitDF(testCsvPath) :
    predSplitDF = dupLR(pd.read_csv(testCsvPath))
    splitAllDupCols(predSplitDF)
    return predSplitDF
def writeFinalDF(testCsvPath, predSplitDF, outCsvPath) :
    outDF = pd.read_csv(testCsvPath)
    nPatients = len(outDF)
    predSplitDFL = predSplitDF[:nPatients].copy().reset_index(drop=True)
    predSplitDFR = predSplitDF[nPatients:].copy().reset_index(drop=True)
    allLabels = {'E':[], 'J':[]}
    for EOrJ in "EJ" :
        for labelListFunc in [HandLabelList,FootLabelList] :
            lLabels,rLabels,labels = [labelListFunc(pref,EOrJ) for pref in ['L','R','']]
            outDF[lLabels] = predSplitDFL[labels]
            outDF[rLabels] = predSplitDFR[labels]
            allLabels[EOrJ].extend(lLabels+rLabels)
    clipPredictions(outDF)
    outDF['Overall_erosion'] = outDF[allLabels['E']].sum(axis='columns')
    outDF['Overall_narrowing'] = outDF[allLabels['J']].sum(axis='columns')
    outDF['Overall_Tol'] = outDF[['Overall_erosion','Overall_narrowing']].sum(axis='columns')
    outDF.to_csv(outCsvPath,index=False)
def predictAllNew(testCsvPath, testImDirPath, outCsvPath) :
    predSplitDF = initPredSplitDF(testCsvPath)
    predictedOrientDF = predictOrient(predSplitDF,'B')
    predictedJointsDF = predictJoints(predSplitDF,'B',
                                       predictedOrientDF=predictedOrientDF)
    # predict all labels
    for name,hPars in pointHPars.items() :
        pointDF = combDFToPointDF(predSplitDF,hPars['trLabel'])
        predictUsingExportedModel(trainedModelPath, name+'_model'+'.pkl', pointDF,
                                  testImDirPath, ['result'],
                                  dict(hPars, trLabel=['result'],
                                       predictedOrientDF=predictedOrientDF,
                                       predictedJointsDF=predictedJointsDF))
        insertPointDF(pointDF,predSplitDF)
    for name,hPars in cutHPars.items() :
        predictUsingExportedModel(trainedModelPath, name+'_model'+'.pkl', predSplitDF,
                                  testImDirPath, hPars['trLabel'],
                                  dict(hPars,
                                       predictedOrientDF=predictedOrientDF,
                                       predictedJointsDF=predictedJointsDF))
    writeFinalDF(testCsvPath, predSplitDF, outCsvPath)

In [ ]:
# export for_prediction
if 'stepAllNew' in running_exported_scripts :
    predictAllNew(imDirPath/mainCsvFName, imDirPath, outDirPath/'predictions.csv')

## Debugging/tuning

In [ ]:
# View examples with worst training and validation errors
# Used to manually check the worst error cases for image regression models.
def getMaxMSE(learn, n=20, ds_type=DatasetType.Valid) :
    preds,target = learn.get_preds(ds_type=ds_type)
    preds = preds.flatten()
    errs = sorted(zip(range(len(preds)),(preds-target)**2,preds,target),
                  key=lambda x : x[1], reverse=True)
    return [(learn.data.dl(ds_type).dataset[i][0], err.item(), pred.item(), targ.item())
           for i,err,pred,targ in errs[:n]]
    return errs
def plotMaxMSEs(learn, ns=3, figsize=(9,9), ds_type=DatasetType.Valid) :
    maxMSEs = getMaxMSE(learn, ns*ns, ds_type)
    fig,axes = plt.subplots(ns, ns, figsize=figsize)
    for i,(im,err,pred,targ) in enumerate(maxMSEs) :
        im.show(ax=axes.flat[i],
               title=f'{"^" if pred>targ else "v"} pred {pred:.3f}, act {targ:.3f}')

In [ ]:
def mymse(x,y) :
    return ((x.flatten()-y.flatten())**2).mean()
def get_mse(learn) :
    pred,target = learn.get_preds()
    return mymse(pred,target)
def get_mse_and_pearson(learn,col=None) :
    pred,target = learn.get_preds()
    # print(pred.shape,target.shape)
    if col is not None :
        pred = pred[:,col]
        target = target[:,col]
    return mymse(pred,target), pearsonr(pred,target)

def get_TTA_mse(learn, nRuns=4, beta=0.4) :
    pred,target = learn.get_preds()
    saveTfms = learn.data.valid_ds.tfms
    learn.data.valid_ds.tfms = learn.data.train_ds.tfms
    for i in range(nRuns) :
        p,_ = learn.get_preds()
        if i == 0 :
            ttaPred = p
        else :
            ttaPred += p
        print(i,'-',((p.flatten()-target.flatten())**2).mean())
    learn.data.valid_ds.tfms = saveTfms
    ttaPred /= nRuns
    pred = pred*beta + ttaPred*(1-beta)
    return mymse(pred,target), pearsonr(pred,target)

def getMetsTr(name,suff,valSliceNo=0,**kwargs) :
    trainUsingPredictedJoints(setupTraining,**dict(pointHPars[name],valSliceNo=valSliceNo,**kwargs))
    learn.load(name+suff)
    return get_mse_and_pearson(learn)
def getMetsCu(name,suff,**kwargs) :
    trainUsingCutJoint(setupTraining,**dict(cutHPars[name],**kwargs))
    learn.load(name+suff)
    return get_mse_and_pearson(learn)

In [ ]:
def checkPredictionsDotCsv(predFPath=None) :
    """Check the predictions.csv file produced by rapredtestAllNew.py"""
    if predFPath is None :
        predFPath = imDirPath/'predictions.csv'
    testPred = pd.read_csv(predFPath)
    testPred = testPred[testPred.columns[1:]]
    testTarget = pd.read_csv(imDirPath/'training.csv')
    testTarget = testTarget[testTarget.columns[1:]]
    return sorted(zip(((testPred-testTarget)**2).mean(),testPred.columns), reverse=True)

# checkPredictionsDotCsv(imDirPath/'dock'/'output'/'predictions.csv')

# checkPredictionsDotCsv(imDirPath/'predictions3.csv')

## Run up to here to train orientation and joint location models.
Then, run the four orientation and joint location training code sections below to train and save/export the four models (orientation and joint location for hands and feet).
After doing this, you can train the RA label prediction models.

In [ ]:
# mark1

In [ ]:
predictedDupDFJoints = predictJoints(dupDF,'B')
# predict joint locations on the training set for use in training label prediction

## Run up to here to train RA label prediction models.
This can only be done after joint location models have been exported.

In [ ]:
# mark2

In [ ]:
# This is run to train and export all the models for RA label prediction,
# using all of the training data (no validation split).
fullTrainAllNew()

In [ ]:
# This is run to update the Docker directory with the latest scripts and models
updateDockerDir(str(imDirPath/'dock'))

## Test - run through the prediction pipeline on part of the training set

In [ ]:
def getSmallTestDF() : return pd.concat([mainDF[10:100],mainDF[200:320]]).reset_index(drop=True)
smallTestDF = getSmallTestDF()
smallTestDF[smallTestDF.columns[1:]] = 'notavail'
smallTestDF.to_csv(imDirPath/'smalltest.csv',index=False)
smallTestDF.head()

In [ ]:
predictAllNew(imDirPath/'smalltest.csv',imDirPath,imDirPath/'smalltest_out2.csv')

In [ ]:
smallTestPred = pd.read_csv(imDirPath/'smalltest_out2.csv')
smallTestPred = smallTestPred[smallTestPred.columns[1:]] 
smallTestTarget = getSmallTestDF()
smallTestTarget = smallTestTarget[smallTestTarget.columns[1:]] 

In [ ]:
sorted(zip(((smallTestPred-smallTestTarget)**2).mean(),smallTestPred.columns), reverse=True)

# Orientation and joint location training
The following four sections are run once to train and save/export the models for orientation and joint location for hands and feet.

In [ ]:
# mark3

## Orientation training - hands
This section is run once to train and save/export the hand orientation model.

In [ ]:
fmDF = makeFlipModDF(dupDF)

In [ ]:
setupTraining(df=fmDF, **HOrientTrainPars);

In [ ]:
learn.data.show_batch()

In [ ]:
learn.data.classes

In [ ]:
trainingHPars

In [ ]:
learn.lr_find(); learn.recorder.plot()

In [ ]:
faiutils.randomSeedForTraining(22)
learn.fit_one_cycle(3,0.01)

In [ ]:
learn.save('HOrient_3epoch_0_01_wd_0_1')
learn.export('HOrient_3epoch_0_01_wd_0_1.pkl')

## Orientation training - feet
This section is run once to train and save/export the foot orientation model.

In [ ]:
fmDF = makeFlipModDF(dupDF)

In [ ]:
setupTraining(df=fmDF, **FOrientTrainPars);

In [ ]:
learn.data.show_batch()

In [ ]:
learn.data.classes

In [ ]:
trainingHPars

In [ ]:
learn.lr_find(); learn.recorder.plot()

In [ ]:
faiutils.randomSeedForTraining(22)
learn.fit_one_cycle(3,0.01)

In [ ]:
learn.save('FOrient_3epoch_0_01_wd_0_1')
learn.export('FOrient_3epoch_0_01_wd_0_1.pkl')

## Joint location training using marked points - hands
This section is run once to train and save/export the hand joint location model.
This model predicts the location of 14 joints used in scoring
(10 of these are finger joints used directly,
while 4 are wrist points that are used for the 12 wrist joints used in scoring -
to make the marking easier, I combined 3 wrist joints each into one marked point).

In [ ]:
setupTraining(**HJLocTrainPars);

In [ ]:
data.show_batch(3,figsize=(9,9))

In [ ]:
learn.lr_find(); learn.recorder.plot()

In [ ]:
# tryCrossVal(40,0.04,**HJLocTrainPars)

In [ ]:
faiutils.randomSeedForTraining(22)
learn.fit_one_cycle(40,0.04)

In [ ]:
learn.save('HJoint_40epoch_0_04_wd_0_1')
learn.export('HJoint_40epoch_0_04_wd_0_1.pkl')

## Joint location training using marked points - feet
This section is run once to train and save/export the foot joint location model.
This model predicts the location of 6 toe joints used in scoring.

In [ ]:
setupTraining(**FJLocTrainPars);

In [ ]:
data.show_batch(3,figsize=(9,9))

In [ ]:
learn.lr_find(); learn.recorder.plot()

In [ ]:
# tryCrossVal(40,0.04,**FJLocTrainPars)

In [ ]:
faiutils.randomSeedForTraining(22)
learn.fit_one_cycle(40,0.04)

In [ ]:
learn.save('FJoint_40epoch_0_04_wd_0_1')
learn.export('FJoint_40epoch_0_04_wd_0_1.pkl')

In [ ]:
# mark4

## Checking the joint location prediction:
This section was used for a manual check on how good the predicted joint locations were.

In [ ]:
check_validation = True  # check on validation only or on the whole dataset
HOrF = 'H'  # check on hands or feet

def setupJointHPars() :
    if HOrF=='F' :
        setupTraining(**FJLocTrainPars)
    else :
        setupTraining(**HJLocTrainPars)

def dfToTensor(df,labelList) :
    df = df[labelList]
    return tensor(df.values)

if check_validation :
    # check predictions of one model after training, using the validation set
    setupJointHPars()
    modelName = HOrF+'Joint_40epoch_0_04_wd_0_1'
    learn.load(modelName)
    preds,target = learn.get_preds()
    vpids = [os.path.basename(fp) for fp in data.valid_ds.items]
else :
    # check all hand or foot predictions on the whole dataset including training and validation sets,
    # using previously trained exported models
    labelList = addYX(allMarkedJointLabels(HOrF))
    predictedJointsDF = predictJoints(dupDF,HOrF)
    setupJointHPars()
    target = dfToTensor(pd.read_csv(HOrF+'joints.csv'), labelList)
    target = target*2.0 - 1.0  # rescale from 0..1 to the -1..1 range trained for by the NN
    vpids = list(predictedJointsDF['Patient_ID'])
    preds = dfToTensor(predictedJointsDF,labelList)

print('MSE ', ((target.flatten()-preds.flatten())**2).mean().item())

il = RAImageList.from_df(dupDF,imDirPath)
il[0];  # to initialize il.jdf


In [ ]:
selectID = widgets.Select(options=vpids, rows=10)
selectDisp = widgets.Select(options=['original marked',
                                     'target (should be same as marked)',
                                     'predicted'], rows=10)
def updateIm() :
    if selectDisp.value.startswith('target') :
        pts = target[selectID.index]
    elif selectDisp.value.startswith('predicted') :
        pts = preds[selectID.index]
    else :
        pts = None
    trainingHPars['debugMarkPts'] = pts
    showPoints(il,selectID.value,pts)
print('Select an image and target/predicted, then execute the next cell to see it.')
display(widgets.AppLayout(left_sidebar=selectID, right_sidebar=selectDisp))

In [ ]:
plt.clf()
updateIm()

## Debugging the joint location labelling:
This section was used to debug feeding the marked joint locations into the Fastai training pipeline.

### Part 1 - untransformed images:

In [ ]:
trainingHPars = dict(defTrainingHPars, trHOrF='F',
                     #trJoints=['F_mtp_XX__2','F_mtp_XX__ip'],
                     trJoints=allMarkedJointLabels('F'),
                     resizeToSize=raGuiDispImSize)
il = RAImageList.from_df(dupDF,imDirPath)

In [ ]:
img = il[0]

In [ ]:
img

In [ ]:
img.size

In [ ]:
showPoints(il,0)

In [ ]:
getJointPoints(il,'UAB722R')

In [ ]:
showPoints(il,'UAB722R')

### Part 2 - transformed images:

In [ ]:
tfms = get_transforms(do_flip=False,flip_vert=False,
                         max_rotate=trainingHPars['max_rotate'],
                         max_lighting=trainingHPars['max_lighting'],
                         max_warp=trainingHPars['max_warp'],
                         max_zoom=trainingHPars['max_zoom'],
                         )
data = (il
         .split_by_rand_pct(0.2,42)
         .label_from_func(partial(getJointPoints,il),label_cls=PointsLabelList)
         .transform(tfms, tfm_y=True, size=256)
         .databunch().normalize(imagenet_stats)
       )

# code from the Biwi head pose example:
# data = (PointsItemList.from_folder(path)
#         .split_by_valid_func(lambda o: o.parent.name=='13')
#         .label_from_func(get_ctr)
#         .transform(get_transforms(), tfm_y=True, size=(120,160))
#         .databunch().normalize(imagenet_stats)
#        )

In [ ]:
data.show_batch(3,figsize=(9,9))

# Examples of performance testing/experimenting

## Using cross-validation:

In [ ]:
# Testing hand joint narrowing prediction (all pip finger joints)
trainUsingPredictedJoints(tryCrossVal,**dict(pointHPars['handTopJ'],
                                              # can try different hyperparameter values here: gBlur=3, etc.
                                             ))

In [ ]:
# Testing hand joint narrowing prediction (all mcp finger joints)
trainUsingPredictedJoints(tryCrossVal,**dict(pointHPars['handMidJ'],
                                              # can try different hyperparameter values here: gBlur=3, etc.
                                             ))

In [ ]:
# Testing wrist narrowing prediction (cmc3, cmc4, cmc5)
trainUsingCutJoint(tryCrossVal,**dict(cutHPars['wrist1'],
                                              # can try different hyperparameter values here: gBlur=3, etc.
                                             ))

In [ ]:
# Testing wrist narrowing prediction (mna, radcar, capnlun)
trainUsingCutJoint(tryCrossVal,**dict(cutHPars['wrist2'],
                                              # can try different hyperparameter values here: gBlur=3, etc.
                                             ))

In [ ]:
# Testing foot joint narrowing prediction (all joints)
trainUsingPredictedJoints(tryCrossVal,**dict(pointHPars['footJ'],
                                              # can try different hyperparameter values here: gBlur=3, etc.
                                             ))

In [ ]:
# Testing hand erosion prediction (all finger joints)
trainUsingPredictedJoints(tryCrossVal,**dict(pointHPars['handTopAndMidE'],
                                              # can try different hyperparameter values here: gBlur=3, etc.
                                             ))

In [ ]:
# Testing wrist erosion prediction (nav, mul, mc1)
trainUsingCutJoint(tryCrossVal,**dict(cutHPars['wrist3'],
                                              # can try different hyperparameter values here: gBlur=3, etc.
                                             ))

In [ ]:
# Testing wrist erosion prediction (ulna, lunate, radius)
trainUsingCutJoint(tryCrossVal,**dict(cutHPars['wrist4'],
                                              # can try different hyperparameter values here: gBlur=3, etc.
                                             ))

In [ ]:
# Testing foot erosion prediction (all joints)
trainUsingPredictedJoints(tryCrossVal,**dict(pointHPars['footE'],
                                              # can try different hyperparameter values here: gBlur=3, etc.
                                             ))

## Experimenting on only one cross-validation split:

In [ ]:
# Testing wrist narrowing prediction (cmc3, cmc4, cmc5)
trainUsingCutJoint(tryTraining,**dict(cutHPars['wrist1'],
                                              # can try different hyperparameter values here: gBlur=3, etc.
                                             ))

In [ ]:
# Testing hand erosion prediction (all finger joints)
trainUsingPredictedJoints(tryTraining,**dict(pointHPars['handTopAndMidE'],
                                              # can try different hyperparameter values here: gBlur=3, etc.
                                             ))